In [1]:
import subprocess
import sys
import os

# Print current working directory
print("Current working directory:", os.getcwd())

# Install from the correct project path
project_path = "c:\\Users\\seros\\OneDrive\\Desktop\\end-to-end-ml-project"
result = subprocess.run([sys.executable, "-m", "pip", "install", "-e", project_path], 
                      capture_output=True, text=True)
print(result.stdout)
if result.returncode != 0:
    print(f"Error: {result.stderr}")

Current working directory: c:\Users\seros\OneDrive\Desktop\end-to-end-ml-project\research
Obtaining file:///C:/Users/seros/OneDrive/Desktop/end-to-end-ml-project
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for ML_Project (pyproject.toml): started
  Building editable for ML_Project (pyproject.toml): finished with status 'done'
  Created wheel for ML_Project: filename=ml_project-0.0.0-0.editable-py3-none-any.whl size=2439 sha256=d9b2a9b349d6887b384769da3b837dbc199eae932a3ae6c324828e29ac44049a
  Stored in directory: C:\Users\

In [2]:
import os

In [2]:
%pwd

'c:\\Users\\seros\\OneDrive\\Desktop\\end-to-end-ml-project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\seros\\OneDrive\\Desktop\\end-to-end-ml-project'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [5]:
from src.ML_Project.constants import *

In [6]:
from src.ML_Project.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH,
    ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir),
        )

        return data_ingestion_config    


    

In [10]:
import os
import urllib.request as request
import zipfile
from ML_Project.utils import logger
from ML_Project.utils.common import get_size


In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        try:
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            logger.info(f"Extracted files to {unzip_path}")
        except (zipfile.BadZipFile, OSError) as e:
            logger.error(f"Invalid or corrupted zip file. Removing and re-downloading...")
            os.remove(self.config.local_data_file)
            self.download_file()
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            logger.info(f"Extracted files to {unzip_path}")


In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2026-02-07 15:09:15,923: INFO: common : yaml file: config\config.yaml loaded successfully]
[2026-02-07 15:09:15,926: INFO: common : yaml file: params.yaml loaded successfully]
[2026-02-07 15:09:15,928: INFO: common : yaml file: schema.yaml loaded successfully]
[2026-02-07 15:09:15,930: INFO: common : created directory at: artifacts]
[2026-02-07 15:09:15,932: INFO: common : created directory at: artifacts/data_ingestion]
[2026-02-07 15:09:17,450: INFO: 4194141443 : artifacts\data_ingestion\data.zip download! with following info: 
Connection: close
Content-Length: 25166
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "4fc42b54ff88754d0438667ac96c3cd5a8e7775a723b142dddacef4605231577"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 5DCC:93D51:88A1A:185F36:69870845
Accept-Ranges: bytes
Date: Sat